<a href="https://colab.research.google.com/github/abhinavtk1/CS6910-A2/blob/main/main_part_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MA23M002 - ABHINAV T K
# CS6910 - Assignment 2

**Github repo:** https://github.com/abhinavtk1/CS6910-A2 <br>
**Wandb report:**

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import matplotlib.image as mpimg
import wandb
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import random_split

In [ ]:
train = 'train/'
test = 'val/'

In [ ]:
classes = ['Amphibia','Animalia','Arachnida','Aves','Fungi',
           'Insecta','Mammalia','Mollusca','Plantae','Reptilia']
IMG_SIZE = (128,128)

In [ ]:
# visualization of each classes
fig = plt.figure(figsize=(18,9))
r, c = 2, 5
l = 1

for item in classes:
  loc = os.path.join(test, item)
  for img in os.listdir(loc):
    img_loc=(os.path.join(loc,img))
    img = mpimg.imread(img_loc)
    img = cv2.resize(img,IMG_SIZE)
    fig.add_subplot(r, c, l)
    plt.imshow(img)
    plt.title(item)
    l+=1
    break

# Part A: Training from scratch


Question 1

Define model

In [ ]:
# Data preparation

# Connect to google drive
#from google.colab import drive
#drive.mount('/content/drive')
from torchvision import transforms, datasets
train_dir = 'train'
test_dir = 'val'
IMG_SIZE = (128,128)
batch_size = 64

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])           # Convert images to PyTorch tensors
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_dir, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = datasets.ImageFolder(root=test_dir, transform=transform)




In [ ]:
# Reference: https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/

class CNN(nn.Module):
    def __init__(self, trainset, valset, batch_size = 32, num_filters = [32,64,64,128,128], kernel_size = [3]*5,
                 act_fn = 'relu', num_neurons = 512):
        super().__init__()

        # DataLoader for the dataset
        self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
        self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)
        #dataloader_test = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=True)

        #Conv layer 1
        self.conv1 = nn.Conv2d(3, num_filters[0],
                               kernel_size=(kernel_size[0],kernel_size[0]),
                               stride=1, padding='same')
        #x = (128 - kernel_size[0] + 2)/1 + 1
        x = 128           # img size
        if act_fn=='relu':          #nn.ReLU nn.GELU nn.SiLU nn.Mish nn.Tanh
          self.act1 = nn.ReLU()
        elif act_fn=='tanh':
          self.act1 = nn.Tanh()
        elif act_fn=='gelu':
          self.act1 = nn.GELU()
        elif act_fn=='silu':
          self.act1 = nn.SiLU()
        elif act_fn=='mish':
          self.act1 = nn.Mish()
        self.pool1 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 2
        self.conv2 = nn.Conv2d(num_filters[0], num_filters[1],
                               kernel_size=(kernel_size[1],kernel_size[1]),
                               stride=1, padding='same')
        #x = (x - kernel_size[1] + 2)/1 + 1

        if act_fn=='relu':
          self.act2 = nn.ReLU()
        elif act_fn=='tanh':
          self.act2 = nn.Tanh()
        elif act_fn=='gelu':
          self.act2 = nn.GELU()
        elif act_fn=='silu':
          self.act2 = nn.SiLU()
        elif act_fn=='mish':
          self.act2 = nn.Mish()
        self.pool2 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 3
        self.conv3 = nn.Conv2d(num_filters[1], num_filters[2],
                               kernel_size=(kernel_size[2],kernel_size[2]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act3 = nn.ReLU()
        elif act_fn=='tanh':
          self.act3 = nn.Tanh()
        elif act_fn=='gelu':
          self.act3 = nn.GELU()
        elif act_fn=='silu':
          self.act3 = nn.SiLU()
        elif act_fn=='mish':
          self.act3 = nn.Mish()
        self.pool3 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 4
        self.conv4 = nn.Conv2d(num_filters[2], num_filters[3],
                               kernel_size=(kernel_size[3],kernel_size[3]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act4 = nn.ReLU()
        elif act_fn=='tanh':
          self.act4 = nn.Tanh()
        elif act_fn=='gelu':
          self.act4 = nn.GELU()
        elif act_fn=='silu':
          self.act4 = nn.SiLU()
        elif act_fn=='mish':
          self.act4 = nn.Mish()
        self.pool4 = nn.MaxPool2d(2,2)
        x = x//2

        #Conv layer 5
        self.conv5 = nn.Conv2d(num_filters[3], num_filters[4],
                               kernel_size=(kernel_size[4],kernel_size[4]),
                               stride=1, padding='same')

        if act_fn=='relu':
          self.act5 = nn.ReLU()
        elif act_fn=='tanh':
          self.act5 = nn.Tanh()
        elif act_fn=='gelu':
          self.act5 = nn.GELU()
        elif act_fn=='silu':
          self.act5 = nn.SiLU()
        elif act_fn=='mish':
          self.act5 = nn.Mish()
        self.pool5 = nn.MaxPool2d(2,2)
        x = x//2
        x = x*x*num_filters[4]
        #print(x)

        # Flatten
        self.flatten = nn.Flatten()

        # dense layer
        self.fc6 = nn.Linear(x, num_neurons)
        if act_fn=='relu':
          self.act6 = nn.ReLU()
        elif act_fn=='tanh':
          self.act6 = nn.Tanh()
        elif act_fn=='gelu':
          self.act6 = nn.GELU()
        elif act_fn=='silu':
          self.act6 = nn.SiLU()
        elif act_fn=='mish':
          self.act6 = nn.Mish()
        # output layer
        self.fc7 = nn.Linear(num_neurons, 10)


    def forward(self, x):
        # input 3x128x128, output 32x128x128
        x = self.act1(self.conv1(x))
        # input 32x128x128, output 32x64x64
        x = self.pool1(x)

        # input 32x64x64, output 32x64x64
        x = self.act2(self.conv2(x))
        # input 32x64x64, output 32x32x32
        x = self.pool2(x)

        # input 32x32x32, output 64x32x32
        x = self.act3(self.conv3(x))
        # input 64x32x32, output 64x16x16
        x = self.pool3(x)

        # input 64x16x16, output 64x16x16
        x = self.act4(self.conv4(x))
        # input 64x16x16, output 64x8x8
        x = self.pool4(x)

        # input 64x8x8, output 128x8x8
        x = self.act5(self.conv5(x))
        # input 128x8x8, output 128x4x4
        x = self.pool5(x)

        # input 128,4,4 output 2048
        x = self.flatten(x)

        # input 2048, output 512
        x = self.act6(self.fc6(x))

        # input 512, output 10
        x = self.fc7(x)
        return x

    def train(self, model, loss_fn, optimizer, n_epochs = 10):
        for epoch in range(n_epochs):
            train_accuracy = 0
            count = 0
            for inputs, labels in self.dataloader_train:
                # forward, backward, and then weight update
                inputs = inputs #.to(device)
                labels = labels #.to(device)
                # Forward pass
                y_pred = model(inputs)
                loss = loss_fn(y_pred, labels)
                train_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
                count += len(labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            train_accuracy /= count
            print("Epoch %d: model train accuracy %.2f%%" % (epoch, train_accuracy*100))
            wandb.log({ 'epoch': epoch, 'train_accuracy': train_accuracy * 100})

            val_accuracy = 0
            count = 0
            for inputs, labels in self.dataloader_val:
                inputs = inputs #.to(device)
                labels = labels #.to(device)
                y_pred = model(inputs)
                val_loss = loss_fn(y_pred, labels)
                val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
                count += len(labels)
            val_accuracy /= count
            print("Epoch %d: model validation accuracy %.2f%%" % (epoch, val_accuracy*100))
            wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})
            wandb.log({ 'epoch': epoch, 'validation_loss': val_loss * 100})

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cpu


Q2. Training the inaturalist dataset

In [ ]:
# sweep config file
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep - test2',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'filters_num':{
            'values': [[32]*5,[32,32,64,64,128],[32,64,64,128,128],[64]*5]
        },
        'act_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish', 'tanh'] # nn.ReLU nn.GELU nn.SiLU nn.Mish nn.Tanh
        },
        'data_augmentation': {          # add
            'values': ['yes'] #,'no']
        },
        'batch_normalisation':{       # add
            'values': ['yes'] #,'no']
        },
        'dropout': {                  # add
            'values': [0] #, 0.2, 0.3]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {               # add
            'values': [32, 64, 128]
        },
        'kernel_size':{
            'values': [[3]*5, [3, 5, 5, 7, 7], [5]*5, [7]*7, [7, 5, 5, 3, 3] ]
        },
        'num_neurons_dense':{
            'values': [128, 256, 512] #, 1028]
        },
        'epochs':{
            'values': [10] #, 20, 30]
        }
    }
}

In [ ]:
# Create a sweep
sweep_id = wandb.sweep(sweep = sweep_config, entity="abhinavtk", project='a2_demo')

# Export data to wandb
#wandb.log({ 'epoch': epoch, 'loss': cost, 'accuracy': accuracy * 100})
#wandb.log({ 'epoch': epoch, 'validation_loss': val_cost, 'validation_accuracy': val_accuracy * 100})
# 4966bf774ea7ce7b47a6e7360b1fad927ece6f1c

Create sweep with ID: hrmc7gn3
Sweep URL: https://wandb.ai/abhinavtk/a2_demo/sweeps/hrmc7gn3


In [ ]:
def main():
  with wandb.init() as run:
    run_name = "-#fil_"+str(wandb.config.filters_num)+"-ac_fn_"+wandb.config.act_fn+"-d_aug_"\
                  +wandb.config.data_augmentation + "-norm_"+wandb.config.batch_normalisation + "-bs_"\
                  +str(wandb.config.batch_size) + "-lr_"+str(wandb.config.learning_rate)+\
                  "-ep_"+str(wandb.config.epochs)

    wandb.run.name = run_name

    model = CNN(trainset, valset, num_filters = wandb.config.filters_num, kernel_size = wandb.config.kernel_size,
                 act_fn = wandb.config.act_fn, num_neurons = wandb.config.num_neurons_dense)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
    model.train(model, loss_fn, optimizer, n_epochs = wandb.config.epochs)
wandb.agent(sweep_id, function = main, count = 50) # calls main function for count number of times
wandb.finish()

wandb: Agent Starting Run: cet1myl7 with config:
wandb: 	act_fn: gelu
wandb: 	batch_normalisation: yes
wandb: 	batch_size: 64
wandb: 	data_augmentation: yes
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	filters_num: [32, 64, 64, 128, 128]
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	learning_rate: 0.001
wandb: 	num_neurons_dense: 256


In [ ]:
model = CNN(trainset, valset, act_fn = 'silu',  num_neurons = 128)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
model.train(model, loss_fn, optimizer, n_epochs = 5)

Epoch 0: model train accuracy 10.01%
Epoch 0: model accuracy 9.95%
Epoch 1: model train accuracy 10.01%
Epoch 1: model accuracy 9.95%
Epoch 2: model train accuracy 10.01%
Epoch 2: model accuracy 9.95%
Epoch 3: model train accuracy 10.01%
Epoch 3: model accuracy 9.95%
Epoch 4: model train accuracy 10.01%
Epoch 4: model accuracy 9.95%


In [ ]:
# sweep config file
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep - test1',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'filters_num':{
            'values': [[32]*5,[32,32,64,64,128],[32,64,64,128,128],[64]*5]
        },
        'act_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish', 'tanh'] # nn.ReLU nn.GELU nn.SiLU nn.Mish nn.Tanh
        },
        'data_augmentation': {          # add
            'values': ['yes','no']
        },
        'batch_normalisation':{       # add
            'values': ['yes','no']
        },
        'dropout': {                  # add
            'values': [0, 0.2, 0.3]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'batch_size': {               # add
            'values': [32, 64, 128]
        },
        'kernel_size':{
            'values': [[3]*5, [3, 5, 5, 7, 7], [5]*5, [7]*7, [7, 5, 5, 3, 3] ]
        },
        'num_neurons_dense':{
            'values': [128, 256, 512] #, 1028]
        },
        'epochs':{
            'values': [10] #, 20, 30]
        }
    }
}

In [ ]:
def main():
  with wandb.init() as run:
    run_name = "-#fil_"+str(wandb.config.filters_num)+"-ac_fn_"+wandb.config.act_fn+"-d_aug_"\
                  +wandb.config.data_augmentation + "-norm_"+wandb.config.batch_normalisation + "-bs_"\
                  +str(wandb.config.batch_size) + "-lr_"+str(wandb.config.learning_rate)+\
                  "-ep_"+str(wandb.config.epochs)

    wandb.run.name = run_name

    model = CNN(act_fn = 'silu',  num_neurons = 128)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
    model.train(model, loss_fn, optimizer, dataloader_train, dataloader_val, n_epochs = 5)
wandb.agent(sweep_id, function = main, count = 5) # calls main function for count number of times
wandb.finish()

wandb: Agent Starting Run: 33nzzizy with config:
wandb: 	act_fn: relu
wandb: 	batch_normalisation: yes
wandb: 	batch_size: 32
wandb: 	data_augmentation: yes
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	filters_num: [32, 64, 64, 128, 128]
wandb: 	kernel_size: [5, 5, 5, 5, 5]
wandb: 	learning_rate: 0.0001
wandb: 	num_neurons_dense: 128
wandb: Currently logged in as: abhinavtk. Use `wandb login --relogin` to force relogin


Epoch 0: model accuracy 9.91%
Epoch 1: model accuracy 9.91%
Epoch 2: model accuracy 9.91%
Epoch 3: model accuracy 9.91%
